In [1]:
import pandas as pd

from oversampling import oversampling_multiclass
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder

# CLASSIFICAZIONE MULTICLASSE

In [2]:
csv_file_path = 'C:/underwater-classification/features_extraction/audio_features.csv'

In [3]:
df = pd.read_csv(csv_file_path)

In [4]:
df = df[df['Class'] == 'Target'].copy()

In [5]:
df.to_csv("audio_features_target.csv", index=False)

In [6]:
print("Distribuzione delle subclass prima dello split:")
print(df['Subclass'].value_counts())

Distribuzione delle subclass prima dello split:
Subclass
Passengership                       11728
Tanker                              11221
Tug                                 10162
Cargo                                9671
Vessel                                307
Sonar                                  20
Seismic airguns surveys                18
Seal bomb                              17
Echosounder                            14
Underwater Communication Signals        6
Scuba divers                            4
Seafloor fossil fuel processing         4
Acoustic release                        3
Acoustic Harassment Devices             2
Explosion                               2
Fishing pinger                          2
Name: count, dtype: int64


In [7]:
df

,File Name,Class,Subclass,Spectral Centroid Mean,Spectral Bandwidth RMS,Standard Deviation,Skewness,Kurtosis,Shannon Entropy,Renyi Entropy,Rate of Attack,Rate of Decay,Threshold Crossings,Silence Ratio,Mean,Max Over Mean,Min Over Mean,Energy Measurements
7812,AHD_Airmar_seg1.wav,Target,Acoustic Harassment Devices,4477.358083,4252.395919,0.052583,-0.065835,23.613643,3.186076,-7.698312,362.326087,0.009282,23776,0.698419,-0.000042,-17164.64500,16960.625000,0.002765
7813,AHD_Airmar_seg2.wav,Target,Acoustic Harassment Devices,2623.478284,2626.760521,0.041744,-0.095743,81.076911,2.165237,-8.520670,228.392857,0.007694,13548,0.815664,-0.000165,-5967.37500,5275.948000,0.001743
7814,SanctSound_CI04_04_acousticrelease_20200202T07...,Target,Acoustic release,3509.880580,5613.858656,0.060189,1.121935,145.730186,2.333120,-7.910301,338.500000,0.003327,1912,0.815846,-0.000454,-2200.42000,2200.487300,0.003623
7815,SanctSound_CI04_04_acousticrelease_20200202T07...,Target,Acoustic release,2521.859074,5201.373090,0.067112,0.740150,134.817244,2.341261,-7.763672,421.000000,0.004587,4383,0.757898,0.004102,243.77046,-243.777900,0.004521
7816,SanctSound_CI04_04_acousticrelease_20200202T07...,Target,Acoustic release,1209.973836,2659.503763,0.043428,1.686207,264.717755,1.846858,-8.548927,345.000000,0.003038,1761,0.909339,-0.001581,-632.46875,632.488040,0.001888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50988,ship_small_seg5.wav,Target,Vessel,456.747011,797.837269,0.194804,0.013841,0.040806,4.912163,-5.505589,374.087500,0.004186,3257,0.502427,0.010898,67.17063,-71.718500,0.038067
50989,ship_small_seg6.wav,Target,Vessel,452.009912,804.062723,0.187102,0.015867,0.072808,4.923598,-5.597314,374.125000,0.004034,3284,0.499966,0.010380,68.85387,-69.641780,0.035115
50990,ship_small_seg7.wav,Target,Vessel,446.388360,808.959976,0.179071,0.000758,0.094906,4.904681,-5.670921,374.100000,0.003832,3247,0.500477,0.011149,60.26118,-65.437140,0.032191
50991,ship_small_seg8.wav,Target,Vessel,451.316540,794.817133,0.188441,-0.002943,0.136695,4.846811,-5.513616,374.087500,0.004043,3283,0.495841,0.010464,76.87661,-72.440750,0.035620


# SPLIT

In [8]:
X_train_imputed, X_val_imputed, X_test_imputed, y_train_encoded, y_val_encoded, y_test_encoded, subclass_encoder = oversampling_multiclass.split_dataset(df)

Dimensione totale dopo il filtraggio: 43089 campioni

Dimensione del set di addestramento: 33872 campioni (78.61%)
Dimensione del set di validazione: 4403 campioni (10.22%)
Dimensione del set di test: 4814 campioni (11.17%)

Distribuzione delle subclass nel set di addestramento:
Subclass
Passengership    8830
Tanker           8744
Tug              8213
Cargo            7802
Vessel            283
Name: count, dtype: int64

Distribuzione delle subclass nel set di validazione:
Subclass
Passengership    1291
Tanker           1276
Tug               956
Cargo             862
Vessel             18
Name: count, dtype: int64

Distribuzione delle subclass nel set di test:
Subclass
Passengership    1607
Tanker           1201
Cargo            1007
Tug               993
Vessel              6
Name: count, dtype: int64


In [9]:
X_val_imputed.to_csv("X_train.csv", index=False)
X_train_imputed[['File Name', 'Parent', 'Subclass']].groupby(['Parent', 'File Name']).describe()

Subclass  \
                                                                                       count   
Parent                               File Name                                                 
1cc5583b-fe72-4cb2-b64c-563bd87ad91b 1cc5583b-fe72-4cb2-b64c-563bd87ad91b_seg1.wav         1   
                                     1cc5583b-fe72-4cb2-b64c-563bd87ad91b_seg10.wav        1   
                                     1cc5583b-fe72-4cb2-b64c-563bd87ad91b_seg11.wav        1   
                                     1cc5583b-fe72-4cb2-b64c-563bd87ad91b_seg12.wav        1   
                                     1cc5583b-fe72-4cb2-b64c-563bd87ad91b_seg13.wav        1   
...                                                                                      ...   
ship_small                           ship_small_seg5.wav                                   1   
                                     ship_small_seg6.wav                                   1   
                                     ship_small_seg7.wav                                   1   
                                     ship_small_seg8.wav                                   1   
                                     ship_small_seg9.wav                                   1   

                                                                                            \
                                                                                    unique   
Parent                               File Name                                               
1cc5583b-fe72-4cb2-b64c-563bd87ad91b 1cc5583b-fe72-4cb2-b64c-563bd87ad91b_seg1.wav       1   
                                     1cc5583b-fe72-4cb2-b64c-563bd87ad91b_seg10.wav      1   
                                     1cc5583b-fe72-4cb2-b64c-563bd87ad91b_seg11.wav      1   
                                     1cc5583b-fe72-4cb2-b64c-563bd87ad91b_seg12.wav      1   
                                     1cc5583b-fe72-4cb2-b64c-563bd87ad91b_seg13.wav      1   
...                                                                                    ...   
ship_small                           ship_small_seg5.wav                                 1   
                                     ship_small_seg6.wav                                 1   
                                     ship_small_seg7.wav                                 1   
                                     ship_small_seg8.wav                                 1   
                                     ship_small_seg9.wav                                 1   

                                                                                                    \
                                                                                               top   
Parent                               File Name                                                       
1cc5583b-fe72-4cb2-b64c-563bd87ad91b 1cc5583b-fe72-4cb2-b64c-563bd87ad91b_seg1.wav   Passengership   
                                     1cc5583b-fe72-4cb2-b64c-563bd87ad91b_seg10.wav  Passengership   
                                     1cc5583b-fe72-4cb2-b64c-563bd87ad91b_seg11.wav  Passengership   
                                     1cc5583b-fe72-4cb2-b64c-563bd87ad91b_seg12.wav  Passengership   
                                     1cc5583b-fe72-4cb2-b64c-563bd87ad91b_seg13.wav  Passengership   
...                                                                                            ...   
ship_small                           ship_small_seg5.wav                                    Vessel   
                                     ship_small_seg6.wav                                    Vessel   
                                     ship_small_seg7.wav                                    Vessel   
                                     ship_small_seg8.wav                                    Vessel   
                                     ship_small_seg9.wav                                    Vess

In [10]:
X_test_imputed.to_csv("X_test.csv", index=False)
X_test_imputed[['File Name', 'Parent','Subclass']].groupby(['Parent', 'File Name']).describe()

Subclass  \
                                                                                                    count   
Parent                                        File Name                                                     
20160516a-9_9                                 20160516a-9_9_seg1.wav                                    1   
                                              20160516a-9_9_seg2.wav                                    1   
                                              20160516a-9_9_seg3.wav                                    1   
20160616b-21_200904                           20160616b-21_200904_seg1.wav                              1   
                                              20160616b-21_200904_seg10.wav                             1   
...                                                                                                   ...   
SanctSound_GR01_01_vessel_20181217T204109Z    SanctSound_GR01_01_vessel_20181217T204109Z_seg2...        1   
SanctSound_HI04_02_smallboat_20191211T175807Z SanctSound_HI04_02_smallboat_20191211T175807Z_s...        1   
                                              SanctSound_HI04_02_smallboat_20191211T175807Z_s...        1   
SanctSound_SB03_09_vessel_20200415T161235Z    SanctSound_SB03_09_vessel_20200415T161235Z_seg1...        1   
                                              SanctSound_SB03_09_vessel_20200415T161235Z_seg2...        1   

                                                                                                         \
                                                                                                 unique   
Parent                                        File Name                                                   
20160516a-9_9                                 20160516a-9_9_seg1.wav                                  1   
                                              20160516a-9_9_seg2.wav                                  1   
                                              20160516a-9_9_seg3.wav                                  1   
20160616b-21_200904                           20160616b-21_200904_seg1.wav                            1   
                                              20160616b-21_200904_seg10.wav                           1   
...                                                                                                 ...   
SanctSound_GR01_01_vessel_20181217T204109Z    SanctSound_GR01_01_vessel_20181217T204109Z_seg2...      1   
SanctSound_HI04_02_smallboat_20191211T175807Z SanctSound_HI04_02_smallboat_20191211T175807Z_s...      1   
                                              SanctSound_HI04_02_smallboat_20191211T175807Z_s...      1   
SanctSound_SB03_09_vessel_20200415T161235Z    SanctSound_SB03_09_vessel_20200415T161235Z_seg1...      1   
                                              SanctSound_SB03_09_vessel_20200415T161235Z_seg2...      1   

                                                                                                                 \
                                                                                                            top   
Parent                                        File Name                                                           
20160516a-9_9                                 20160516a-9_9_seg1.wav                              Passengership   
                                              20160516a-9_9_seg2.wav                              Passengership   
                                              20160516a-9_9_seg3.wav                              Passengership   
20160616b-21_200904                           20160616b-21_200904_seg1.wav                        Passengership   
                                              20160616b-21_200904_seg10.wav                       Passengership   
...                                                                                                         ...   
SanctSound_GR01_01_vessel_20181217T2

In [11]:
X_val_imputed.to_csv("X_val.csv", index=False)
X_val_imputed[['File Name', 'Parent','Subclass']].groupby(['Parent', 'File Name']).describe()


Subclass  \
                                                                                              count   
Parent                                   File Name                                                    
20160531-7_142749                        20160531-7_142749_seg1.wav                               1   
                                         20160531-7_142749_seg10.wav                              1   
                                         20160531-7_142749_seg100.wav                             1   
                                         20160531-7_142749_seg101.wav                             1   
                                         20160531-7_142749_seg102.wav                             1   
...                                                                                             ...   
SanctSound_OC02_01_ship_20190419T233040Z SanctSound_OC02_01_ship_20190419T233040Z_seg5.wav        1   
                                         SanctSound_OC02_01_ship_20190419T233040Z_seg6.wav        1   
                                         SanctSound_OC02_01_ship_20190419T233040Z_seg7.wav        1   
                                         SanctSound_OC02_01_ship_20190419T233040Z_seg8.wav        1   
                                         SanctSound_OC02_01_ship_20190419T233040Z_seg9.wav        1   

                                                                                                   \
                                                                                           unique   
Parent                                   File Name                                                  
20160531-7_142749                        20160531-7_142749_seg1.wav                             1   
                                         20160531-7_142749_seg10.wav                            1   
                                         20160531-7_142749_seg100.wav                           1   
                                         20160531-7_142749_seg101.wav                           1   
                                         20160531-7_142749_seg102.wav                           1   
...                                                                                           ...   
SanctSound_OC02_01_ship_20190419T233040Z SanctSound_OC02_01_ship_20190419T233040Z_seg5.wav      1   
                                         SanctSound_OC02_01_ship_20190419T233040Z_seg6.wav      1   
                                         SanctSound_OC02_01_ship_20190419T233040Z_seg7.wav      1   
                                         SanctSound_OC02_01_ship_20190419T233040Z_seg8.wav      1   
                                         SanctSound_OC02_01_ship_20190419T233040Z_seg9.wav      1   

                                                                                                    \
                                                                                               top   
Parent                                   File Name                                                   
20160531-7_142749                        20160531-7_142749_seg1.wav                         Tanker   
                                         20160531-7_142749_seg10.wav                        Tanker   
                                         20160531-7_142749_seg100.wav                       Tanker   
                                         20160531-7_142749_seg101.wav                       Tanker   
                                         20160531-7_142749_seg102.wav                       Tanker   
...                                                                                            ...   
SanctSound_OC02_01_ship_20190419T233040Z SanctSound_OC02_01_ship_20190419T233040Z_seg5.wav  Vessel   
                                         SanctSound_OC02_01_ship_20190419T233040Z_seg6.wav  Vessel   
                                         SanctSound_OC02_01_ship_20190419T233040Z_seg7.wav  Vessel   
         

# SMOTE

In [ ]:
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "4"  # Adatta questo valore al numero di core che desideri utilizzare

In [ ]:
X_train_resampled, y_train_resampled = oversampling_multiclass.apply_smote_multiclass(X_train_imputed, y_train_encoded, 1)

In [ ]:
print(pd.Series(subclass_encoder.inverse_transform(y_train_resampled)).value_counts())

In [ ]:
print(f"\nDistribuzione delle classi nel set di validation:\n \n{pd.Series(y_val_encoded).value_counts()} ")

In [ ]:

print(f"\nDistribuzione delle classi nel set di test:\n \n{pd.Series(y_test_encoded).value_counts()} ")

In [ ]:
# Stampare le forme dei dataset
print("\nForme dei dataset:")
print(f"X_train_imputed: {X_train_imputed.shape}") #validation
print(f"X_val_imputed: {X_val_imputed.shape}") #validation
print(f"X_test_imputed: {X_test_imputed.shape}") #test
print(f"X_train_resampled: {X_train_resampled.shape}") #train dopo SMOTE

# Esperimento 1 

Esperimento effettuato prendendo come riferimento alcune features numeriche da "Malfante et al."

In [ ]:
# Rimuovi le colonne non necessarie per evitare errori nei nomi delle feature
X_val_imputed = X_val_imputed.drop(columns=["Class", "File Name", "Parent", "Subclass"], errors='ignore')
X_test_imputed = X_test_imputed.drop(columns=["Class", "File Name", "Parent", "Subclass"], errors='ignore')


In [ ]:
random_forest_model = oversampling_multiclass.train_random_forest_multiclass(X_train_resampled, y_train_resampled, X_val_imputed, y_val_encoded, X_test_imputed, y_test_encoded)

In [ ]:
# Predizioni del modello per il set di validazione
y_val_pred = random_forest_model.predict(X_val_imputed)

# Predizioni del modello per il set di test
y_test_pred = random_forest_model.predict(X_test_imputed)


# Genera la matrice di confusione per il set di validazione
cm_val = confusion_matrix(y_val_encoded, y_val_pred, labels=np.unique(y_val_encoded))
disp_val = ConfusionMatrixDisplay(confusion_matrix=cm_val, display_labels=np.unique(y_val_encoded))

# Genera la matrice di confusione per il set di test
cm_test = confusion_matrix(y_test_encoded, y_test_pred, labels=np.unique(y_test_encoded))
disp_test = ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=np.unique(y_test_encoded))

# Crea una figura con due sotto-grafici affiancati
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Mostra la matrice di confusione per il set di validazione
disp_val.plot(cmap='Blues', ax=axes[0])
axes[0].set_title("Random Forest - Matrice di Confusione (Validazione)")

# Mostra la matrice di confusione per il set di test
disp_test.plot(cmap='Blues', ax=axes[1])
axes[1].set_title("Random Forest - Matrice di Confusione (Test)")

# Rimuove la griglia
for ax in axes:
    ax.grid(False)

# Mostra entrambe le matrici di confusione
plt.tight_layout()
plt.show()


In [ ]:
svm_model = oversampling_multiclass.train_svm_multiclass(X_train_resampled, y_train_resampled, X_val_imputed, y_val_encoded, X_test_imputed, y_test_encoded)

In [ ]:
# Predizioni del modello per il set di validazione
y_val_pred = svm_model.predict(X_val_imputed)

# Predizioni del modello per il set di test
y_test_pred = svm_model.predict(X_test_imputed)

# Genera la matrice di confusione per il set di validazione
cm_val = confusion_matrix(y_val_encoded, y_val_pred, labels=np.unique(y_val_encoded))
disp_val = ConfusionMatrixDisplay(confusion_matrix=cm_val, display_labels=np.unique(y_val_encoded))

# Genera la matrice di confusione per il set di test
cm_test = confusion_matrix(y_test_encoded, y_test_pred, labels=np.unique(y_test_encoded))
disp_test = ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=np.unique(y_test_encoded))

# Crea una figura con due sotto-grafici affiancati
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Mostra la matrice di confusione per il set di validazione
disp_val.plot(cmap='Blues', ax=axes[0])
axes[0].set_title("SVM - Matrice di Confusione (Validazione)")

# Mostra la matrice di confusione per il set di test
disp_test.plot(cmap='Blues', ax=axes[1])
axes[1].set_title("SVM - Matrice di Confusione (Test)")

# Rimuove la griglia
for ax in axes:
    ax.grid(False)

# Mostra entrambe le matrici di confusione
plt.tight_layout()
plt.show()

In [ ]:
print("\nDistribuzione delle classi nel set di validazione:")
print(pd.Series(subclass_encoder.inverse_transform(y_val_encoded)).value_counts())

In [ ]:
print(pd.Series(y_val_encoded).value_counts())

In [ ]:
print("\nDistribuzione delle classi predette nel set di validazione:")
print(pd.Series(subclass_encoder.inverse_transform(y_val_pred)).value_counts())

In [ ]:
print(pd.Series(y_val_pred).value_counts())

In [ ]:
lightgbm_model = oversampling_multiclass.train_lightgbm(X_train_resampled, y_train_resampled, X_val_imputed, y_val_encoded, X_test_imputed, y_test_encoded)

In [ ]:
# Predizioni del modello per il set di validazione
y_val_pred_proba = lightgbm_model.predict(X_val_imputed)
# Converti le probabilità in etichette di classe
y_val_pred = np.argmax(y_val_pred_proba, axis=1)

# Predizioni del modello per il set di test
y_test_pred_proba = lightgbm_model.predict(X_test_imputed)
# Converti le probabilità in etichette di classe
y_test_pred = np.argmax(y_test_pred_proba, axis=1)

# Genera la matrice di confusione per il set di validazione
cm_val = confusion_matrix(y_val_encoded, y_val_pred, labels=np.unique(y_val_encoded))
disp_val = ConfusionMatrixDisplay(confusion_matrix=cm_val, display_labels=np.unique(y_val_encoded))

# Genera la matrice di confusione per il set di test
cm_test = confusion_matrix(y_test_encoded, y_test_pred, labels=np.unique(y_test_encoded))
disp_test = ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=np.unique(y_test_encoded))

# Crea una figura con due sotto-grafici affiancati
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Mostra la matrice di confusione per il set di validazione
disp_val.plot(cmap='Blues', ax=axes[0])
axes[0].set_title("LightGBM - Matrice di Confusione (Validazione)")

# Mostra la matrice di confusione per il set di test
disp_test.plot(cmap='Blues', ax=axes[1])
axes[1].set_title("LightGBM - Matrice di Confusione (Test)")

# Rimuove la griglia
for ax in axes:
    ax.grid(False)

# Mostra entrambe le matrici di confusione
plt.tight_layout()
plt.show()



## Esperimento 2

Esperimento effettuato prendendo come riferimento alcune features numeriche da “BirdNet”

In [ ]:
# Caricare il dataset e applicare lo split
csv_file_path = 'features_extraction/exp1_audio_features.csv'
X_train_imputed_exp1, X_val_imputed_exp1, X_test_imputed_exp1, y_train_encoded_exp1, y_val_encoded_exp1, y_test_encoded_exp1, subclass_encoder_exp1 = oversampling_multiclass.split_dataset(csv_file_path)
# Applicare SMOTE
X_train_resampled_exp1, y_train_resampled_exp1 = oversampling_multiclass.apply_smote_multiclass(X_train_imputed_exp1, y_train_encoded_exp1, 1)
#TRAIN
random_forest_model_exp1 = oversampling_multiclass.train_random_forest_multiclass(X_train_resampled_exp1, y_train_resampled_exp1, X_val_imputed_exp1, y_val_encoded_exp1, X_test_imputed_exp1, y_test_encoded_exp1)

In [ ]:
# Predizioni del modello per il set di validazione
y_val_pred_exp1 = random_forest_model_exp1.predict(X_val_imputed_exp1)

# Predizioni del modello per il set di test
y_test_pred_exp1 = random_forest_model_exp1.predict(X_test_imputed_exp1)

# Genera la matrice di confusione per il set di validazione
cm_val_exp1 = confusion_matrix(y_val_encoded_exp1, y_val_pred_exp1, labels=np.unique(y_val_encoded_exp1))
disp_val_exp1 = ConfusionMatrixDisplay(confusion_matrix=cm_val_exp1, display_labels=np.unique(y_val_encoded_exp1))

# Genera la matrice di confusione per il set di test
cm_test_exp1 = confusion_matrix(y_test_encoded_exp1, y_test_pred_exp1, labels=np.unique(y_test_encoded_exp1))
disp_test_exp1 = ConfusionMatrixDisplay(confusion_matrix=cm_test_exp1, display_labels=np.unique(y_test_encoded_exp1))

# Crea una figura con due sotto-grafici affiancati
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Mostra la matrice di confusione per il set di validazione
disp_val_exp1.plot(cmap='Blues', ax=axes[0])
axes[0].set_title("Random Forest - Matrice di Confusione (Validazione)")

# Mostra la matrice di confusione per il set di test
disp_test_exp1.plot(cmap='Blues', ax=axes[1])
axes[1].set_title("Random Forest - Matrice di Confusione (Test)")

# Rimuove la griglia
for ax in axes:
    ax.grid(False)

# Mostra entrambe le matrici di confusione
plt.tight_layout()
plt.show()

In [ ]:
svm_model_exp1 = oversampling_multiclass.train_svm_multiclass(X_train_resampled_exp1, y_train_resampled_exp1, X_val_imputed_exp1, y_val_encoded_exp1, X_test_imputed_exp1, y_test_encoded_exp1)

In [ ]:
# Predizioni del modello per il set di validazione
y_val_pred_exp1 = svm_model_exp1.predict(X_val_imputed_exp1)

# Predizioni del modello per il set di test
y_test_pred_exp1 = svm_model_exp1.predict(X_test_imputed_exp1)

# Genera la matrice di confusione per il set di validazione
cm_val_exp1 = confusion_matrix(y_val_encoded_exp1, y_val_pred_exp1, labels=np.unique(y_val_encoded_exp1))
disp_val_exp1 = ConfusionMatrixDisplay(confusion_matrix=cm_val_exp1, display_labels=np.unique(y_val_encoded_exp1))

# Genera la matrice di confusione per il set di test
cm_test_exp1 = confusion_matrix(y_test_encoded_exp1, y_test_pred_exp1, labels=np.unique(y_test_encoded_exp1))
disp_test_exp1 = ConfusionMatrixDisplay(confusion_matrix=cm_test_exp1, display_labels=np.unique(y_test_encoded_exp1))

# Crea una figura con due sotto-grafici affiancati
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Mostra la matrice di confusione per il set di validazione
disp_val_exp1.plot(cmap='Blues', ax=axes[0])
axes[0].set_title("SVM - Matrice di Confusione (Validazione)")

# Mostra la matrice di confusione per il set di test
disp_test_exp1.plot(cmap='Blues', ax=axes[1])
axes[1].set_title("SVM - Matrice di Confusione (Test)")

# Rimuove la griglia
for ax in axes:
    ax.grid(False)

# Mostra entrambe le matrici di confusione
plt.tight_layout()
plt.show()


In [ ]:
lightgbm_model_exp1 = oversampling_multiclass.train_lightgbm(X_train_resampled_exp1, y_train_resampled_exp1, X_val_imputed_exp1, y_val_encoded_exp1, X_test_imputed_exp1, y_test_encoded_exp1)

In [ ]:
# Predizioni del modello per il set di validazione
y_val_pred_proba_exp1 = lightgbm_model_exp1.predict(X_val_imputed_exp1)
# Converti le probabilità in etichette di classe
y_val_pred_exp1 = np.argmax(y_val_pred_proba_exp1, axis=1)

# Predizioni del modello per il set di test
y_test_pred_proba_exp1 = lightgbm_model_exp1.predict(X_test_imputed_exp1)
# Converti le probabilità in etichette di classe
y_test_pred_exp1 = np.argmax(y_test_pred_proba_exp1, axis=1)

# Genera la matrice di confusione per il set di validazione
cm_val_exp1 = confusion_matrix(y_val_encoded_exp1, y_val_pred_exp1, labels=np.unique(y_val_encoded_exp1))
disp_val_exp1 = ConfusionMatrixDisplay(confusion_matrix=cm_val_exp1, display_labels=np.unique(y_val_encoded_exp1))

# Genera la matrice di confusione per il set di test
cm_test_exp1 = confusion_matrix(y_test_encoded_exp1, y_test_pred_exp1, labels=np.unique(y_test_encoded_exp1))
disp_test_exp1 = ConfusionMatrixDisplay(confusion_matrix=cm_test_exp1, display_labels=np.unique(y_test_encoded_exp1))

# Crea una figura con due sotto-grafici affiancati
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Mostra la matrice di confusione per il set di validazione
disp_val_exp1.plot(cmap='Blues', ax=axes[0])
axes[0].set_title("LightGBM - Matrice di Confusione (Validazione)")

# Mostra la matrice di confusione per il set di test
disp_test_exp1.plot(cmap='Blues', ax=axes[1])
axes[1].set_title("LightGBM - Matrice di Confusione (Test)")

# Rimuove la griglia
for ax in axes:
    ax.grid(False)

# Mostra entrambe le matrici di confusione
plt.tight_layout()
plt.show()

## Esperimento 3

Esperimento effettuato prendendo come riferimento features numeriche da "A Survey on Audio Feature Extraction for Automatic Music Genre Classification" di Dhamodaran et al.

In [ ]:
# Caricare il dataset e applicare lo split
csv_file_path = 'features_extraction/exp2_audio_features.csv'
X_train_imputed_exp2, X_val_imputed_exp2, X_test_imputed_exp2, y_train_encoded_exp2, y_val_encoded_exp2, y_test_encoded_exp2, subclass_encoder_exp2 = oversampling_multiclass.split_dataset(csv_file_path)
# Applicare SMOTE
X_train_resampled_exp2, y_train_resampled_exp2 = oversampling_multiclass.apply_smote_multiclass(X_train_imputed_exp2, y_train_encoded_exp2, 1)
#TRAIN
random_forest_model_exp2 = oversampling_multiclass.train_random_forest_multiclass(X_train_resampled_exp2, y_train_resampled_exp2, X_val_imputed_exp2, y_val_encoded_exp2, X_test_imputed_exp2, y_test_encoded_exp2)

In [ ]:
# Predizioni del modello per il set di validazione
y_val_pred_exp2 = random_forest_model_exp2.predict(X_val_imputed_exp2)

# Predizioni del modello per il set di test
y_test_pred_exp2 = random_forest_model_exp2.predict(X_test_imputed_exp2)

# Genera la matrice di confusione per il set di validazione
cm_val_exp2 = confusion_matrix(y_val_encoded_exp2, y_val_pred_exp2, labels=np.unique(y_val_encoded_exp2))
disp_val_exp2 = ConfusionMatrixDisplay(confusion_matrix=cm_val_exp2, display_labels=np.unique(y_val_encoded_exp2))

# Genera la matrice di confusione per il set di test
cm_test_exp2 = confusion_matrix(y_test_encoded_exp2, y_test_pred_exp2, labels=np.unique(y_test_encoded_exp2))
disp_test_exp2 = ConfusionMatrixDisplay(confusion_matrix=cm_test_exp2, display_labels=np.unique(y_test_encoded_exp2))

# Crea una figura con due sotto-grafici affiancati
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Mostra la matrice di confusione per il set di validazione
disp_val_exp2.plot(cmap='Blues', ax=axes[0])
axes[0].set_title("Random Forest - Matrice di Confusione (Validazione)")

# Mostra la matrice di confusione per il set di test
disp_test_exp2.plot(cmap='Blues', ax=axes[1])
axes[1].set_title("Random Forest - Matrice di Confusione (Test)")

# Rimuove la griglia
for ax in axes:
    ax.grid(False)

# Mostra entrambe le matrici di confusione
plt.tight_layout()
plt.show()

In [ ]:
svm_model_exp2 = oversampling_multiclass.train_svm_multiclass(X_train_resampled_exp2, y_train_resampled_exp2, X_val_imputed_exp2, y_val_encoded_exp2, X_test_imputed_exp2, y_test_encoded_exp2)

In [ ]:
# Predizioni del modello per il set di validazione
y_val_pred_exp2 = svm_model_exp2.predict(X_val_imputed_exp2)

# Predizioni del modello per il set di test
y_test_pred_exp2 = svm_model_exp2.predict(X_test_imputed_exp2)

# Genera la matrice di confusione per il set di validazione
cm_val_exp2 = confusion_matrix(y_val_encoded_exp2, y_val_pred_exp2, labels=np.unique(y_val_encoded_exp2))
disp_val_exp2 = ConfusionMatrixDisplay(confusion_matrix=cm_val_exp2, display_labels=np.unique(y_val_encoded_exp2))

# Genera la matrice di confusione per il set di test
cm_test_exp2 = confusion_matrix(y_test_encoded_exp2, y_test_pred_exp2, labels=np.unique(y_test_encoded_exp2))
disp_test_exp2 = ConfusionMatrixDisplay(confusion_matrix=cm_test_exp2, display_labels=np.unique(y_test_encoded_exp2))

# Crea una figura con due sotto-grafici affiancati
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Mostra la matrice di confusione per il set di validazione
disp_val_exp2.plot(cmap='Blues', ax=axes[0])
axes[0].set_title("SVM - Matrice di Confusione (Validazione)")

# Mostra la matrice di confusione per il set di test
disp_test_exp2.plot(cmap='Blues', ax=axes[1])
axes[1].set_title("SVM - Matrice di Confusione (Test)")

# Rimuove la griglia
for ax in axes:
    ax.grid(False)

# Mostra entrambe le matrici di confusione
plt.tight_layout()
plt.show()

In [ ]:
lightgbm_model_exp2 = oversampling_multiclass.train_lightgbm(X_train_resampled_exp2, y_train_resampled_exp2, X_val_imputed_exp2, y_val_encoded_exp2, X_test_imputed_exp2, y_test_encoded_exp2)

In [ ]:
# Predizioni del modello per il set di validazione
y_val_pred_proba_exp2 = lightgbm_model_exp2.predict(X_val_imputed_exp2)
# Converti le probabilità in etichette di classe
y_val_pred_exp2 = np.argmax(y_val_pred_proba_exp2, axis=1)

# Predizioni del modello per il set di test
y_test_pred_proba_exp2 = lightgbm_model_exp2.predict(X_test_imputed_exp2)
# Converti le probabilità in etichette di classe
y_test_pred_exp2 = np.argmax(y_test_pred_proba_exp2, axis=1)

# Genera la matrice di confusione per il set di validazione
cm_val_exp2 = confusion_matrix(y_val_encoded_exp2, y_val_pred_exp2, labels=np.unique(y_val_encoded_exp2))
disp_val_exp2 = ConfusionMatrixDisplay(confusion_matrix=cm_val_exp2, display_labels=np.unique(y_val_encoded_exp2))

# Genera la matrice di confusione per il set di test
cm_test_exp2 = confusion_matrix(y_test_encoded_exp2, y_test_pred_exp2, labels=np.unique(y_test_encoded_exp2))
disp_test_exp2 = ConfusionMatrixDisplay(confusion_matrix=cm_test_exp2, display_labels=np.unique(y_test_encoded_exp2))

# Crea una figura con due sotto-grafici affiancati
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Mostra la matrice di confusione per il set di validazione
disp_val_exp2.plot(cmap='Blues', ax=axes[0])
axes[0].set_title("LightGBM - Matrice di Confusione (Validazione)")

# Mostra la matrice di confusione per il set di test
disp_test_exp2.plot(cmap='Blues', ax=axes[1])
axes[1].set_title("LightGBM - Matrice di Confusione (Test)")

# Rimuove la griglia
for ax in axes:
    ax.grid(False)

# Mostra entrambe le matrici di confusione
plt.tight_layout()
plt.show()